# Inperpolation Mechanics

In [ ]:
import veloxchem as vlx

Define a VeloxChem Molecule for which the interpolation mechanics force field (IM-MM) shall be constructed.

In [ ]:
# Initialize the Molecu
molecule_xyz = '''16

C          1.35443     0.72389     0.06285
C          0.07503     1.19318    -0.13198
S         -1.06388    -0.07684    -0.23891
C          0.14505    -1.27596    -0.02313
C          1.39245    -0.69978     0.13394
C         -0.16119    -2.68438     0.01602
C          0.34715    -3.61280     0.90741
C         -0.13980    -4.93101     0.66445
C         -0.99297    -4.97247    -0.41303
S         -1.21221    -3.43027    -1.11712
H          2.21964     1.36874     0.15151
H         -0.24847     2.22102    -0.21857
H          2.29991    -1.27380     0.28025
H          1.02874    -3.35748     1.70932
H          0.12522    -5.80285     1.25099
H         -1.50258    -5.83368    -0.82408
'''


molecule = vlx.Molecule.from_xyz_string(molecule_xyz)
molecule.show(atom_indices=False)

Initialize the QM-Driver determining the underlying quality of the interpolation data.

In [ ]:
# Initialize the basis set and the QM driver
basis_set_label = "def2-svp"
basis = vlx.MolecularBasis.read(molecule, basis_set_label)
qm_driver = vlx.ScfRestrictedDriver()
qm_driver.xcfun = "b3lyp"

##### Define dyanmics and interpolation settings which are crucial for the quality in timing of the database. Important variables:

- .desired_point_density -> how many datapoints should the database contain for each conformer.
- .converged_cycle -> after how many cycles the construction for 1 conformer terminates (1 cycle: 3000 steps without database expansion).
- .energy_thrsh -> allowed devaition of the IM energy and QM reference calculations.

##### As this notebook is intended to be a practival example on how to use the IMFroceFieldGenerator but keeping the computational effort at a reasoable level, the shown settings have been used. For production run we recommend using the following settings:
 
 - .nsteps = 50000
 - .snapshots = 10000
 - .desired_point_density = 80
 - .converged_cycle = 5
 - .energy_threshold = 2.0 kcal/mol

In [ ]:
# Initialize the IM force field generator with:
# dihedrals: list of dihedral angles to sample
# sampling_structures: number of structures to sample
# ensemble: NVT or NVE
# temperature: temperature of the ensemble
# nsteps: number of steps to run the dynamics
# snapshots: number of snapshots to save

dyn_drv = vlx.IMForceFieldGenerator(qm_driver)
dyn_drv.dihedrals_dict = [((2, 3, 5, 9), 10)]
dyn_drv.ensemble = 'NVT'
dyn_drv.temperature = 250
dyn_drv.nsteps = 10000
dyn_drv.snapshots = 5000
dyn_drv.converged_cycle = 3
dyn_drv.energy_threshold = 2.0
dyn_drv.desired_point_density = 10
dyn_drv.nstruc_to_confirm_database_quality = 10
dyn_drv.minimize = False


The compute function starts the construction of a new or given database. The construction run ends by confirming the quality of the database and the result are stored.
- note that here we provide the constructed database ('im_database.h5') as the computational cost of the second derivatives are very demanding.

In [ ]:
# Run the database consruction
dyn_drv.imforcefieldfile = 'data/im_database.h5'
im_results = dyn_drv.compute(molecule, basis)

In [ ]:
ff_gen = vlx.MMForceFieldGenerator()
ff_gen.create_topology(molecule)

To use the constructed IM force field one has to initialize the InterpolationDriver() with user-defined settings (recommended). The current implementation is restricted to well seperated IM and MM regions and is not yet included into a linking atom framework.

In [ ]:
z_matrix = dyn_drv.define_z_matrix(molecule)
im_driver = vlx.InterpolationDriver(z_matrix)
interpolation_settings = { 'interpolation_type':'shepard', 
                            'exponent_p':'2',
                            'exponent_q':'2', 
                            'confidence_radius':'0.5',
                            'imforcefieldfile':'im_database.h5'}
im_driver.update_settings(interpolation_settings)

openmmdyn = vlx.OpenMMDynamics()
openmmdyn.create_system_from_molecule(molecule, ff_gen, qm_atoms='all')
openmmdyn.run_qmmm(im_driver, im_driver)
